In [1]:
import nifgen
import numpy as np
import time

# Settings
resource_name = "Dev1"
sample_rate = 30_000_000 
waveform_freq = 7911.24    
duration = 1 / waveform_freq
num_samples = int(sample_rate * duration)

# Timing
t = np.linspace(0, duration, 300, endpoint=False)

# Step 1: Base square wave
square_wave = np.sign(np.sin(2 * np.pi * waveform_freq * t))

# Step 2: Identify high portion (the +1 region)
high_indices = np.where(square_wave > 0)[0]

# Step 3: Create high-frequency pulses (e.g., 1 MHz) to insert into the high part
pulse_freq = 1_000_000  # 1 MHz
pulse_wave = 0.5 * np.sign(np.cos(2 * np.pi * pulse_freq * t[high_indices]))

# Step 4: Replace high part of square wave with high-freq pulses
waveform = square_wave.copy()
waveform[high_indices] = pulse_wave

# Optional: scale from -1/1 to 0/1 if needed
# waveform = (waveform + 1) / 2

waveform = waveform.tolist()

# Initialize session
with nifgen.Session(resource_name) as session:
    # Set to ARB mode
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = sample_rate

    # Write waveform to device memory
    wf_handle = session.create_waveform(waveform)

    # Configure trigger
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_edge = nifgen.StartTriggerDigitalEdgeEdge.RISING
    session.trigger_mode = nifgen.TriggerMode.STEPPED
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"


    session.configure_arb_waveform(wf_handle, gain=1.0, offset=0.0)

    session.initiate()

    print("Armed. A new waveform will play on each rising TTL pulse at PFI1.")
    time.sleep(5)  # Keep session alive for 10 seconds

Armed. A new waveform will play on each rising TTL pulse at PFI1.


In [98]:
waveform_path = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"

In [ ]:
# TTL is too slow for us to work with for every line
# Lets first do it by just the first trigger

